In [1]:
import pandas as pd
import numpy as np

path = '../data/DOGEUSDT_5m.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume
0,2023-08-01 00:00:00,0.07779,0.07781,0.07764,0.07768,16924720.0
1,2023-08-01 00:05:00,0.07768,0.07768,0.07745,0.07761,26522341.0
2,2023-08-01 00:10:00,0.07761,0.07776,0.07760,0.07773,11641017.0
3,2023-08-01 00:15:00,0.07774,0.07786,0.07774,0.07782,11970966.0
4,2023-08-01 00:20:00,0.07782,0.07797,0.07779,0.07781,21013227.0


In [2]:
import pandas_ta as ta

temp = df.ta.bbands()
df['BBANDS_U'] = temp['BBU_5_2.0']
df['BBANDS_L'] = temp['BBL_5_2.0']
df['BBANDS_M'] = temp['BBM_5_2.0']
temp = df.ta.cdl_pattern(name="closingmarubozu")
df['MARUBOZU'] = temp['CDL_CLOSINGMARUBOZU']
#singal_buy = MARUBOZU == 100 and close > DONCH_U 
#singal_sell = MARUBOZU == -100 and close < DONCH_L
df['Signal_Buy'] = np.where((df['MARUBOZU'] == 100) & (df['close'] < df['BBANDS_U']), 1, 0)
df['Signal_Sell'] = np.where((df['MARUBOZU'] == -100) & (df['close'] < df['BBANDS_L']), 1, 0)
df['Signal'] = df['Signal_Buy'].replace(1,2) + df['Signal_Sell']
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

In [3]:
df['Signal'].value_counts()

0    21816
2     1682
1        2
Name: Signal, dtype: int64

In [4]:
df.ta.bbands()

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,0.077572,0.077730,0.077888,0.407804,0.752377
...,...,...,...,...,...
23495,0.060736,0.060836,0.060936,0.329016,0.719824
23496,0.060740,0.060840,0.060940,0.330043,0.549801
23497,0.060747,0.060844,0.060941,0.318289,0.427708
23498,0.060816,0.060864,0.060912,0.158820,0.458620


In [5]:
help(ta.donchian)

Help on function donchian in module pandas_ta.volatility.donchian:

donchian(high, low, lower_length=None, upper_length=None, offset=None, **kwargs)
    Donchian Channels (DC)
    
    Donchian Channels are used to measure volatility, similar to
    Bollinger Bands and Keltner Channels.
    
    Sources:
        https://www.tradingview.com/wiki/Donchian_Channels_(DC)
    
    Calculation:
        Default Inputs:
            lower_length=upper_length=20
        LOWER = low.rolling(lower_length).min()
        UPPER = high.rolling(upper_length).max()
        MID = 0.5 * (LOWER + UPPER)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        lower_length (int): The short period. Default: 20
        upper_length (int): The short period. Default: 20
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type

In [6]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
27,2023-08-01 02:15:00,0.07591,0.07627,0.07583,0.07621,124375358.0,0.077685,0.075467,0.076576,0.0,0,0,0,0.000370,54.054882
28,2023-08-01 02:20:00,0.07620,0.07626,0.07598,0.07598,83717660.0,0.077247,0.075389,0.076318,0.0,0,0,0,0.000364,57.442838
29,2023-08-01 02:25:00,0.07598,0.07621,0.07596,0.07617,34095028.0,0.076410,0.075814,0.076112,0.0,0,0,0,0.000356,60.378550
30,2023-08-01 02:30:00,0.07617,0.07630,0.07609,0.07626,37039197.0,0.076382,0.075826,0.076104,0.0,0,0,0,0.000345,62.282981
31,2023-08-01 02:35:00,0.07627,0.07630,0.07589,0.07615,78194996.0,0.076344,0.075964,0.076154,0.0,0,0,0,0.000350,64.104151


In [7]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_26148\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
time,,,,,,,,,,,,,,
2023-08-01 02:15:00,0.07591,0.07627,0.07583,0.07621,124375358.0,0.077685,0.075467,0.076576,0.0,0,0,0,0.000370,54.054882
2023-08-01 02:20:00,0.07620,0.07626,0.07598,0.07598,83717660.0,0.077247,0.075389,0.076318,0.0,0,0,0,0.000364,57.442838
2023-08-01 02:25:00,0.07598,0.07621,0.07596,0.07617,34095028.0,0.076410,0.075814,0.076112,0.0,0,0,0,0.000356,60.378550
2023-08-01 02:30:00,0.07617,0.07630,0.07609,0.07626,37039197.0,0.076382,0.075826,0.076104,0.0,0,0,0,0.000345,62.282981
2023-08-01 02:35:00,0.07627,0.07630,0.07589,0.07615,78194996.0,0.076344,0.075964,0.076154,0.0,0,0,0,0.000350,64.104151


In [8]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [9]:
df['Signal'].value_counts()

0    21789
2     1682
1        2
Name: Signal, dtype: int64

In [10]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = 350
    profit_target = 500
    size_multiplier_rate = 0.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 2:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == 1:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            

bt = Backtest(df, Scalping_Strategy, cash=350, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-08-01 02:15:00
End                       2023-10-21 14:15:00
Duration                     81 days 12:00:00
Exposure Time [%]                   43.117625
Equity Final [$]                    32.083176
Equity Peak [$]                    550.322297
Return [%]                         -90.833378
Buy & Hold Return [%]              -20.181079
Return (Ann.) [%]                  -99.998104
Volatility (Ann.) [%]                6.235514
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -96.933385
Avg. Drawdown [%]                  -15.920325
Max. Drawdown Duration       76 days 18:05:00
Avg. Drawdown Duration        5 days 19:31:00
# Trades                                 1519
Win Rate [%]                        39.631336
Best Trade [%]                       1.005608
Worst Trade [%]                     -1.454085
Avg. Trade [%]                    

In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [12]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
                    maximize='Equity Final [$]')
stats

C:\Users\hiban\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 576 configurations.



  0%|          | 0/12 [00:00<?, ?it/s]

ValueError: Long orders require: SL (0.061444564831772436) < LIMIT (0.06145122899999999) < TP (0.06145108703364551)

In [ ]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [ ]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=2.0,TPSL_rate=0.2,size_multiplier_rate=0.4)>

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.update_layout(title='Profit |'+symbol, yaxis_title='USDT', xaxis_title='Date')
fig.show()